# Badanie wpływu przetwarzania datasetów na najnowszy model deocelowy SAR

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

model_simple = Sequential([
    Input(shape=(256, 256, 1)),  
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  
])

model_simple.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_simple = model_simple.fit(
    train_generator_sar,  
    validation_data=validation_generator_sar, 
    epochs=20,
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model_simple.evaluate(test_generator_sar)

print(f"Wynik na danych testowych:")
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")




# Nowy model docelowy dla datasetu przetwrozonego metodą wyrównywania histogramu 

In [2]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generators(base_dir, batch_size=32, target_size=(256, 256)):
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'train'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    val_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'val'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    test_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'test'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale',
        shuffle=False  
    )
    
    return train_generator, val_generator, test_generator

def create_model():
    model_simple = Sequential([
    Input(shape=(256, 256, 1)),  
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  
    ])

    model_simple.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model_simple

base_dir_srad = r"C:\Users\piotr\Desktop\PRACA MAGISTERSKA\processed_histogram"  

epochs = 20
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\nTrening dla zestawu przetworzonego metodą wyrównywania histogramu")

train_generator, val_generator, test_generator = create_generators(base_dir_srad, batch_size=batch_size)
model_sar = create_model()
history_sar = model_sar.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model_sar.evaluate(test_generator)
print(f"Wyniki dla metody SRAD: Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")



Trening dla zestawu przetworzonego metodą wyrównywania histogramu
Found 11200 images belonging to 4 classes.
Found 3200 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.
Epoch 1/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 93s 264ms/step - accuracy: 0.3297 - loss: 1.3998 - val_accuracy: 0.5594 - val_loss: 1.0082
Epoch 2/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 87s 248ms/step - accuracy: 0.5817 - loss: 0.9569 - val_accuracy: 0.5716 - val_loss: 0.9776
Epoch 3/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 87s 247ms/step - accuracy: 0.7408 - loss: 0.6510 - val_accuracy: 0.7244 - val_loss: 0.6726
Epoch 4/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 87s 248ms/step - accuracy: 0.8658 - loss: 0.3537 - val_accuracy: 0.7784 - val_loss: 0.5446
Epoch 5/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 87s 250ms/step - accuracy: 0.9507 - loss: 0.1486 - val_accuracy: 0.7722 - val_loss: 0.6407
Epoch 6/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 90s 259ms/step - accuracy: 0.9816 - loss: 0.0678 - val_accuracy: 0.7866 - val_loss: 0.7451
Epoch 7/20
350/350 ━

# Badanie dla datasetu przetwrzonego metodą Gaussa

In [3]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generators(base_dir, batch_size=32, target_size=(256, 256)):
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'train'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    val_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'val'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    test_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'test'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale',
        shuffle=False  
    )
    
    return train_generator, val_generator, test_generator

def create_model():
    model_simple = Sequential([
    Input(shape=(256, 256, 1)),  
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  
    ])

    model_simple.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model_simple

base_dir_srad = r"C:\Users\piotr\Desktop\PRACA MAGISTERSKA\processed_gaussian"  

epochs = 20
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\nTrening dla zestawu przetworzonego metodą Gaussa")

train_generator, val_generator, test_generator = create_generators(base_dir_srad, batch_size=batch_size)
model_sar = create_model()
history_sar = model_sar.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model_sar.evaluate(test_generator)
print(f"Wyniki dla metody Gaussa: Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")



Trening dla zestawu przetworzonego metodą Gaussa
Found 11200 images belonging to 4 classes.
Found 3200 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.
Epoch 1/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 100s 282ms/step - accuracy: 0.3832 - loss: 1.3233 - val_accuracy: 0.4881 - val_loss: 1.1524
Epoch 2/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 95s 271ms/step - accuracy: 0.6320 - loss: 0.8848 - val_accuracy: 0.7991 - val_loss: 0.5230
Epoch 3/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 92s 264ms/step - accuracy: 0.8395 - loss: 0.4310 - val_accuracy: 0.8609 - val_loss: 0.3794
Epoch 4/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 95s 271ms/step - accuracy: 0.9128 - loss: 0.2343 - val_accuracy: 0.8913 - val_loss: 0.2904
Epoch 5/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 91s 260ms/step - accuracy: 0.9556 - loss: 0.1298 - val_accuracy: 0.8522 - val_loss: 0.3739
Epoch 6/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 96s 273ms/step - accuracy: 0.9849 - loss: 0.0600 - val_accuracy: 0.8500 - val_loss: 0.4485
Epoch 7/20
350/350 ━━━━━━━━━━━━━━━━━

In [4]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generators(base_dir, batch_size=32, target_size=(256, 256)):
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'train'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    val_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'val'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    test_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'test'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale',
        shuffle=False  
    )
    
    return train_generator, val_generator, test_generator

def create_model():
    model_simple = Sequential([
    Input(shape=(256, 256, 1)),  
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  
    ])

    model_simple.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model_simple

base_dir_srad = r"C:\Users\piotr\Desktop\PRACA MAGISTERSKA\processed_srad"  

epochs = 20
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\nTrening dla zestawu przetworzonego metodą SRAD")

train_generator, val_generator, test_generator = create_generators(base_dir_srad, batch_size=batch_size)
model_sar = create_model()
history_sar = model_sar.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model_sar.evaluate(test_generator)
print(f"Wyniki dla metody SRAD: Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")



Trening dla zestawu przetworzonego metodą SRAD
Found 11200 images belonging to 4 classes.
Found 3200 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.
Epoch 1/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 106s 299ms/step - accuracy: 0.2955 - loss: 1.4833 - val_accuracy: 0.4934 - val_loss: 1.0980
Epoch 2/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 96s 274ms/step - accuracy: 0.5990 - loss: 0.9173 - val_accuracy: 0.7819 - val_loss: 0.5780
Epoch 3/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 95s 271ms/step - accuracy: 0.8153 - loss: 0.4822 - val_accuracy: 0.8913 - val_loss: 0.3189
Epoch 4/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 95s 271ms/step - accuracy: 0.9243 - loss: 0.2213 - val_accuracy: 0.8813 - val_loss: 0.3098
Epoch 5/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 95s 270ms/step - accuracy: 0.9673 - loss: 0.1074 - val_accuracy: 0.8547 - val_loss: 0.3939
Epoch 6/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 97s 278ms/step - accuracy: 0.9659 - loss: 0.1078 - val_accuracy: 0.8612 - val_loss: 0.3921
Epoch 7/20
350/350 ━━━━━━━━━━━━━━━━━━━

# Wyniki dla datasetu przetworzonego metodą CLAHE

In [1]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generators(base_dir, batch_size=32, target_size=(256, 256)):
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'train'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    val_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'val'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    test_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'test'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale',
        shuffle=False  
    )
    
    return train_generator, val_generator, test_generator

def create_model():
    model_simple = Sequential([
    Input(shape=(256, 256, 1)),  
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  
    ])

    model_simple.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model_simple

base_dir_srad = r"C:\Users\piotr\Desktop\PRACA MAGISTERSKA\processed_clahe"  

epochs = 20
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\nTrening dla zestawu przetworzonego metodą CLAHE")

train_generator, val_generator, test_generator = create_generators(base_dir_srad, batch_size=batch_size)
model_sar = create_model()
history_sar = model_sar.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model_sar.evaluate(test_generator)
print(f"Wyniki dla metody CLAHE: Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

c:\Users\piotr\anaconda_inz\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Trening dla zestawu przetworzonego metodą CLAHE
Found 11200 images belonging to 4 classes.
Found 3200 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.


c:\Users\piotr\anaconda_inz\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 95s 267ms/step - accuracy: 0.3523 - loss: 1.3821 - val_accuracy: 0.5334 - val_loss: 1.0584
Epoch 2/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 85s 242ms/step - accuracy: 0.6953 - loss: 0.7412 - val_accuracy: 0.7450 - val_loss: 0.6321
Epoch 3/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 85s 243ms/step - accuracy: 0.8300 - loss: 0.4422 - val_accuracy: 0.8531 - val_loss: 0.3857
Epoch 4/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 86s 245ms/step - accuracy: 0.9208 - loss: 0.2317 - val_accuracy: 0.8506 - val_loss: 0.3927
Epoch 5/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 85s 244ms/step - accuracy: 0.9675 - loss: 0.1019 - val_accuracy: 0.8363 - val_loss: 0.4377
Epoch 6/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 85s 244ms/step - accuracy: 0.9917 - loss: 0.0450 - val_accuracy: 0.8144 - val_loss: 0.5916
Epoch 7/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 86s 244ms/step - accuracy: 0.9927 - loss: 0.0295 - val_accuracy: 0.7259 - val_loss: 0.8972
Epoch 8/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 85s 244ms/step - accuracy: 0.9732 - loss: 0

# Wyniki dla datasetu przetworzonego metodą sharpened

In [2]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generators(base_dir, batch_size=32, target_size=(256, 256)):
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'train'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    val_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'val'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )
    
    test_generator = datagen.flow_from_directory(
        os.path.join(base_dir, 'test'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale',
        shuffle=False  
    )
    
    return train_generator, val_generator, test_generator

def create_model():
    model_simple = Sequential([
    Input(shape=(256, 256, 1)),  
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  
    ])

    model_simple.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model_simple

base_dir_srad = r"C:\Users\piotr\Desktop\PRACA MAGISTERSKA\processed_sharpened"  

epochs = 20
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\nTrening dla zestawu przetworzonego metodą sharpened")

train_generator, val_generator, test_generator = create_generators(base_dir_srad, batch_size=batch_size)
model_sar = create_model()
history_sar = model_sar.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model_sar.evaluate(test_generator)
print(f"Wyniki dla metody sharpened: Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Trening dla zestawu przetworzonego metodą sharpened
Found 11200 images belonging to 4 classes.
Found 3200 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.
Epoch 1/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 99s 282ms/step - accuracy: 0.4132 - loss: 1.3439 - val_accuracy: 0.6681 - val_loss: 0.7901
Epoch 2/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 88s 251ms/step - accuracy: 0.7396 - loss: 0.6582 - val_accuracy: 0.8222 - val_loss: 0.4365
Epoch 3/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 86s 246ms/step - accuracy: 0.8653 - loss: 0.3790 - val_accuracy: 0.9122 - val_loss: 0.2497
Epoch 4/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 86s 246ms/step - accuracy: 0.9290 - loss: 0.1985 - val_accuracy: 0.9275 - val_loss: 0.2054
Epoch 5/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 88s 251ms/step - accuracy: 0.9578 - loss: 0.1207 - val_accuracy: 0.9481 - val_loss: 0.1604
Epoch 6/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 88s 251ms/step - accuracy: 0.9875 - loss: 0.0415 - val_accuracy: 0.9381 - val_loss: 0.1757
Epoch 7/20
350/350 ━━━━━━━━━━━━━━━

Przetwarzanie zdjęć optycznych 

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

import os

datagen_train = ImageDataGenerator(rescale=1./255)
datagen_val = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)

base_dir_optical = r'C:\Users\piotr\Desktop\projekt_WUM_SAR\OPTICAL'

print("Dla zbioru treningowego:")
train_generator_optical = datagen_train.flow_from_directory(
    os.path.join(base_dir_optical, 'train'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

print("Dla zbioru walidacyjnego:")
validation_generator_optical = datagen_val.flow_from_directory(
    os.path.join(base_dir_optical, 'val'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

print("Dla zbioru testowego:")
test_generator_optical = datagen_test.flow_from_directory(
    os.path.join(base_dir_optical, 'test'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False
)

model_simple = Sequential([
    Input(shape=(256, 256, 3)),  
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  
])

model_simple.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model_simple.fit(
    train_generator_optical,
    validation_data=validation_generator_optical,
    epochs=20,
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model_simple.evaluate(test_generator_optical)
print(f"Wyniki dla metody zdjęć optycznych: Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Dla zbioru treningowego:
Found 11200 images belonging to 4 classes.
Dla zbioru walidacyjnego:
Found 3200 images belonging to 4 classes.
Dla zbioru testowego:
Found 1600 images belonging to 4 classes.
Epoch 1/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 125s 353ms/step - accuracy: 0.6961 - loss: 0.8346 - val_accuracy: 0.9066 - val_loss: 0.2612
Epoch 2/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 111s 316ms/step - accuracy: 0.9442 - loss: 0.1555 - val_accuracy: 0.9222 - val_loss: 0.1970
Epoch 3/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 107s 306ms/step - accuracy: 0.9664 - loss: 0.0920 - val_accuracy: 0.9609 - val_loss: 0.1184
Epoch 4/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 105s 301ms/step - accuracy: 0.9862 - loss: 0.0435 - val_accuracy: 0.9594 - val_loss: 0.1307
Epoch 5/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 103s 294ms/step - accuracy: 0.9895 - loss: 0.0349 - val_accuracy: 0.9419 - val_loss: 0.2205
Epoch 6/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 117s 334ms/step - accuracy: 0.9887 - loss: 0.0357 - val_accuracy: 0.9669 - val_loss: 0.1141
Epoch 7/